**Phase 1:** Python Data Generation Script

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import sqlite3
import random

# Set seed for reproducibility
np.random.seed(42)
random.seed(42)

# ============================================================
# CONFIGURATION: Mine Site Profiles
# ============================================================

sites = {
    'Site A': {
        'name': 'Site A',
        'location': 'Pilbara, WA',
        'target_aisc': 185,  # Bottom-quartile cost producer
        'ore_grade_avg': 2.8,  # % grade
        'tonnes_monthly_avg': 85000,
        'cost_profile': 'low'
    },
    'Site B': {
        'name': 'Site B',
        'location': 'Goldfields, WA',
        'target_aisc': 210,  # High-cost producer (budget overrun)
        'ore_grade_avg': 2.1,  # Lower grade = higher processing costs
        'tonnes_monthly_avg': 65000,
        'cost_profile': 'high'
    },
    'Site C': {
        'name': 'Site C',
        'location': 'Mid-West, WA',
        'target_aisc': 195,  # Medium-cost producer
        'ore_grade_avg': 2.5,
        'tonnes_monthly_avg': 75000,
        'cost_profile': 'medium'
    }
}

# Cost category baselines ($ per tonne)
cost_baselines = {
    'low': {
        'Mining': 68,
        'Processing': 85,
        'G&A': 18,
        'Sustaining CapEx': 14
    },
    'medium': {
        'Mining': 72,
        'Processing': 92,
        'G&A': 19,
        'Sustaining CapEx': 12
    },
    'high': {
        'Mining': 78,
        'Processing': 98,
        'G&A': 22,
        'Sustaining CapEx': 12
    }
}

# ============================================================
# FUNCTION: Generate Monthly Cost Data
# ============================================================

def generate_mining_cost_data(start_date, num_months=24):
    """
    Generate realistic mining cost data with:
    - Seasonal variations (Q1 cyclone season +8% costs)
    - Ore grade impact on processing costs
    - Budget vs. Actual variance
    - Random operational fluctuations
    """

    data_rows = []

    for site_key, site_info in sites.items():
        site_name = site_info['name']
        cost_profile = site_info['cost_profile']
        base_costs = cost_baselines[cost_profile]
        target_aisc = site_info['target_aisc']
        base_ore_grade = site_info['ore_grade_avg']
        base_tonnes = site_info['tonnes_monthly_avg']

        for month_offset in range(num_months):
            # Generate date
            current_date = start_date + timedelta(days=30 * month_offset)
            year = current_date.year
            month = current_date.month
            quarter = (month - 1) // 3 + 1

            # Seasonal adjustment: Q1 (Jan-Mar) has +8% costs due to cyclone season
            seasonal_factor = 1.08 if quarter == 1 else 1.0

            # Ore grade variation: +/- 15% from baseline
            ore_grade = base_ore_grade * np.random.uniform(0.88, 1.12)

            # Ore grade impact on processing costs: Lower grade = higher processing $/tonne
            # 1% grade reduction = +6% processing cost increase
            grade_impact_factor = (base_ore_grade / ore_grade) ** 0.8

            # Tonnes produced: +/- 10% monthly variation
            tonnes_produced = base_tonnes * np.random.uniform(0.92, 1.08)

            # Generate costs for each category
            for category, base_cost_per_tonne in base_costs.items():

                # Apply seasonal factor
                cost_per_tonne = base_cost_per_tonne * seasonal_factor

                # Processing costs affected by ore grade
                if category == 'Processing':
                    cost_per_tonne *= grade_impact_factor

                # Random operational variance: +/- 8%
                cost_per_tonne *= np.random.uniform(0.94, 1.06)

                # Calculate total cost for the month
                total_cost = cost_per_tonne * tonnes_produced

                # Budget: Set at baseline with small adjustments
                budget_cost_per_tonne = base_cost_per_tonne * np.random.uniform(0.98, 1.02)
                budget_total = budget_cost_per_tonne * base_tonnes

                # Create row
                data_rows.append({
                    'SiteID': site_key.replace(' ', ''),
                    'SiteName': site_name,
                    'Location': site_info['location'],
                    'Date': current_date.strftime('%Y-%m-%d'),
                    'Year': year,
                    'Month': month,
                    'Quarter': quarter,
                    'CostCategory': category,
                    'TonnesProduced': round(tonnes_produced, 2),
                    'OreGrade': round(ore_grade, 2),
                    'CostPerTonne_Actual': round(cost_per_tonne, 2),
                    'CostPerTonne_Budget': round(budget_cost_per_tonne, 2),
                    'TotalCost_Actual': round(total_cost, 2),
                    'TotalCost_Budget': round(budget_total, 2),
                    'CostVariance': round(total_cost - budget_total, 2),
                    'VariancePercent': round(((total_cost - budget_total) / budget_total) * 100, 2)
                })

    return pd.DataFrame(data_rows)

# ============================================================
# GENERATE DATA
# ============================================================

start_date = datetime(2024, 1, 1)
df_costs = generate_mining_cost_data(start_date, num_months=24)

print("=" * 80)
print("MINING COST DATA GENERATION - SUMMARY")
print("=" * 80)
print(f"Total Records Generated: {len(df_costs):,}")
print(f"Date Range: {df_costs['Date'].min()} to {df_costs['Date'].max()}")
print(f"Number of Sites: {df_costs['SiteID'].nunique()}")
print(f"Cost Categories: {', '.join(df_costs['CostCategory'].unique())}")
print("\n" + "=" * 80)
print("SAMPLE DATA (First 10 Rows)")
print("=" * 80)
print(df_costs.head(10).to_string(index=False))

# ============================================================
# CALCULATE AISC BY SITE (Validation Check) - CORRECTED
# ============================================================

print("\n" + "=" * 80)
print("AISC VALIDATION CHECK (Overall Average by Site)")
print("=" * 80)

# Calculate total costs and tonnes per site (summing across ALL categories and months)
aisc_summary = df_costs.groupby('SiteName').agg({
    'TotalCost_Actual': 'sum',      # Sum all costs across all categories and months
    'TonnesProduced': 'first',      # Tonnes are duplicated per category, take first value
    'OreGrade': 'mean'              # Average ore grade
}).reset_index()

# Calculate total tonnes properly (sum unique month/site combinations only)
tonnes_by_site = df_costs.groupby(['SiteName', 'Date'])['TonnesProduced'].first().reset_index()
tonnes_summary = tonnes_by_site.groupby('SiteName')['TonnesProduced'].sum().reset_index()
tonnes_summary.columns = ['SiteName', 'TotalTonnes']

# Merge corrected tonnes
aisc_summary = aisc_summary.drop('TonnesProduced', axis=1)
aisc_summary = aisc_summary.merge(tonnes_summary, on='SiteName')

# Calculate AISC correctly
aisc_summary['AISC'] = (aisc_summary['TotalCost_Actual'] / aisc_summary['TotalTonnes']).round(2)
aisc_summary['AvgOreGrade'] = aisc_summary['OreGrade'].round(2)
aisc_summary = aisc_summary[['SiteName', 'AISC', 'AvgOreGrade', 'TotalTonnes', 'TotalCost_Actual']]
aisc_summary['TotalTonnes'] = aisc_summary['TotalTonnes'].round(0).astype(int)
aisc_summary['TotalCost_Actual'] = aisc_summary['TotalCost_Actual'].round(0).astype(int)

print(aisc_summary.to_string(index=False))
print("\nTarget AISC Ranges:")
print("  Site A: $185/t (Low-cost producer)")
print("  Site B: $210/t (High-cost producer)")
print("  Site C: $195/t (Medium-cost producer)")

# Validation status
print("\nValidation Status:")
for _, row in aisc_summary.iterrows():
    site = row['SiteName']
    aisc = row['AISC']
    if site == 'Site A':
        target = 185
    elif site == 'Site B':
        target = 210
    else:
        target = 195

    variance = abs(aisc - target)
    status = "✓ PASS" if variance <= 15 else "✗ FAIL"
    print(f"  {site}: ${aisc}/t vs ${target}/t target → {status} (variance: ${variance:.2f}/t)")

# ============================================================
# SAVE TO CSV (For Review)
# ============================================================

output_csv = 'mining_cost_data.csv'
df_costs.to_csv(output_csv, index=False)
print(f"\n✓ Data saved to: {output_csv}")

# ============================================================
# CREATE SQLITE DATABASE WITH STAR SCHEMA
# ============================================================

db_name = 'mining_aisc_costs.db'
conn = sqlite3.connect(db_name)

# --- Dimension: DimSite ---
dim_site = df_costs[['SiteID', 'SiteName', 'Location']].drop_duplicates().reset_index(drop=True)
dim_site.index.name = 'SiteKey'
dim_site.reset_index(inplace=True)
dim_site.to_sql('DimSite', conn, if_exists='replace', index=False)

# --- Dimension: DimDate ---
dim_date = df_costs[['Date', 'Year', 'Month', 'Quarter']].drop_duplicates().reset_index(drop=True)
dim_date['Date'] = pd.to_datetime(dim_date['Date'])
dim_date['MonthName'] = dim_date['Date'].dt.strftime('%B')
dim_date['YearMonth'] = dim_date['Date'].dt.strftime('%Y-%m')
dim_date.index.name = 'DateKey'
dim_date.reset_index(inplace=True)
dim_date['Date'] = dim_date['Date'].dt.strftime('%Y-%m-%d')
dim_date.to_sql('DimDate', conn, if_exists='replace', index=False)

# --- Dimension: DimCostCategory ---
dim_cost_category = pd.DataFrame({
    'CostCategoryID': ['MINING', 'PROCESSING', 'GA', 'CAPEX'],
    'CostCategoryName': ['Mining', 'Processing', 'G&A', 'Sustaining CapEx'],
    'CostType': ['Operating', 'Operating', 'Operating', 'Capital']
})
dim_cost_category.index.name = 'CostCategoryKey'
dim_cost_category.reset_index(inplace=True)
dim_cost_category.to_sql('DimCostCategory', conn, if_exists='replace', index=False)

# --- Fact: FactCosts ---
fact_costs = df_costs.copy()

# Create foreign keys by merging with dimensions
fact_costs = fact_costs.merge(
    dim_site[['SiteKey', 'SiteID']],
    on='SiteID',
    how='left'
)

fact_costs = fact_costs.merge(
    dim_date[['DateKey', 'Date']],
    on='Date',
    how='left'
)

# Map cost categories
category_map = {
    'Mining': 'MINING',
    'Processing': 'PROCESSING',
    'G&A': 'GA',
    'Sustaining CapEx': 'CAPEX'
}
fact_costs['CostCategoryID'] = fact_costs['CostCategory'].map(category_map)

fact_costs = fact_costs.merge(
    dim_cost_category[['CostCategoryKey', 'CostCategoryID']],
    on='CostCategoryID',
    how='left'
)

# Select fact table columns
fact_costs_final = fact_costs[[
    'SiteKey', 'DateKey', 'CostCategoryKey',
    'TonnesProduced', 'OreGrade',
    'CostPerTonne_Actual', 'CostPerTonne_Budget',
    'TotalCost_Actual', 'TotalCost_Budget',
    'CostVariance', 'VariancePercent'
]]

fact_costs_final.index.name = 'CostID'
fact_costs_final.reset_index(inplace=True)
fact_costs_final.to_sql('FactCosts', conn, if_exists='replace', index=False)

conn.close()

print(f"✓ SQLite database created: {db_name}")
print("\nStar Schema Structure:")
print("  - DimSite (3 sites)")
print("  - DimDate (24 months)")
print("  - DimCostCategory (4 categories)")
print("  - FactCosts (288 records)")

print("\n" + "=" * 80)
print("PHASE 1 COMPLETE: Data generation and database creation successful")
print("=" * 80)
print("\nNext Steps:")
print("  1. Review mining_cost_data.csv for data quality")
print("  2. Validate AISC calculations match targets")
print("  3. Proceed to Phase 2: SQL Queries (cost allocation, variance analysis)")

MINING COST DATA GENERATION - SUMMARY
Total Records Generated: 288
Date Range: 2024-01-01 to 2025-11-21
Number of Sites: 3
Cost Categories: Mining, Processing, G&A, Sustaining CapEx

SAMPLE DATA (First 10 Rows)
SiteID SiteName    Location       Date  Year  Month  Quarter     CostCategory  TonnesProduced  OreGrade  CostPerTonne_Actual  CostPerTonne_Budget  TotalCost_Actual  TotalCost_Budget  CostVariance  VariancePercent
 SiteA   Site A Pilbara, WA 2024-01-01  2024      1        1           Mining        91129.71      2.72                75.48                68.27        6878882.42        5802809.84    1076072.57            18.54
 SiteA   Site A Pilbara, WA 2024-01-01  2024      1        1       Processing        91129.71      2.72                90.19                83.83        8218974.63        7125582.42    1093392.21            15.34
 SiteA   Site A Pilbara, WA 2024-01-01  2024      1        1              G&A        91129.71      2.72                18.41                18.26     

In [4]:
# ============================================================
# DOWNLOAD ALL FILES FOR GITHUB
# ============================================================

from google.colab import files
import os

print("=" * 80)
print("DOWNLOADING PROJECT FILES FOR GITHUB")
print("=" * 80)

# List of files to download
files_to_download = [
    'mining_cost_data.csv',
    'mining_aisc_costs.db',
    'query1_aisc_by_site.csv',
    'query2_cost_breakdown.csv',
    'query3_monthly_trend.csv',
    'query4_budget_variance.csv',
    'query5_quarterly_summary.csv',
    'query6_sensitivity_analysis.csv',
    'query7_quartile_ranking.csv',
    'query8_opex_vs_capex.csv'
]

# Download each file
for file in files_to_download:
    if os.path.exists(file):
        print(f"✓ Downloading: {file}")
        files.download(file)
    else:
        print(f"✗ File not found: {file}")

print("\n" + "=" * 80)
print("DOWNLOAD COMPLETE")
print("=" * 80)
print("\nFiles downloaded. Add these to your GitHub repository:")
print("  /data/mining_cost_data.csv")
print("  /data/mining_aisc_costs.db")
print("  /data/query_outputs/query1_aisc_by_site.csv")
print("  /data/query_outputs/query2_cost_breakdown.csv")
print("  /data/query_outputs/[...remaining query files]")

DOWNLOADING PROJECT FILES FOR GITHUB
✓ Downloading: mining_cost_data.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✓ Downloading: mining_aisc_costs.db


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✗ File not found: query1_aisc_by_site.csv
✗ File not found: query2_cost_breakdown.csv
✗ File not found: query3_monthly_trend.csv
✗ File not found: query4_budget_variance.csv
✗ File not found: query5_quarterly_summary.csv
✗ File not found: query6_sensitivity_analysis.csv
✗ File not found: query7_quartile_ranking.csv
✗ File not found: query8_opex_vs_capex.csv

DOWNLOAD COMPLETE

Files downloaded. Add these to your GitHub repository:
  /data/mining_cost_data.csv
  /data/mining_aisc_costs.db
  /data/query_outputs/query1_aisc_by_site.csv
  /data/query_outputs/query2_cost_breakdown.csv
  /data/query_outputs/[...remaining query files]
